In [54]:
import numpy as np
import pandas as pd

In [55]:
df1 = pd.read_csv("modeling data/1_players_info_added_Clean_data.csv")

df1_nulls = df1.isnull().sum()

print(df1_nulls)

Unnamed: 0           0
Location             0
Date                 0
Round                0
Winner               0
Loser                0
WRank               60
LRank              206
W_hand               0
W_ht               957
W_nationality      858
W_age              859
L_hand               0
L_ht              1102
L_nationality      791
L_age              796
L_wt             11818
W_wt             11801
dtype: int64


In [56]:
df1_dropped_wt = df1.copy()

df1_dropped_wt.drop(columns=["Unnamed: 0",'L_wt', 'W_wt'], inplace=True)
df1_no_nulls = df1_dropped_wt.dropna()

len(df1_no_nulls)
# df1_no_nulls


17917

In [57]:
df_winner = df1_no_nulls.copy()
df_loser = df1_no_nulls.copy()

df_winner['win'] = 1
df_loser['win'] = 0

# Renaming columns to 'player' and 'opponent' for winners
df_winner.rename(columns={
    'Winner': 'player',
    'Loser': 'opponent',
    'WRank': 'player_rank',
    'LRank': 'opponent_rank',
    'W_hand': 'player_hand',
    'L_hand': 'opponent_hand',
    'W_ht': 'player_ht', 
    'W_nationality':'player_nationality', 
    'W_age':'player_age', 
    'L_ht':'opponent_ht',
    'L_nationality':'opponent_nationality', 
    'L_age':'opponent_age'
}, inplace=True)

# Renaming columns to 'player' and 'opponent' for losers
df_loser.rename(columns={
    'Winner': 'opponent',
    'Loser': 'player',
    'WRank': 'opponent_rank',
    'LRank': 'player_rank',
    'W_hand': 'opponent_hand',
    'L_hand': 'player_hand',
    'W_ht':'opponent_ht', 
    'W_nationality':'opponent_nationality', 
    'W_age':'opponent_age', 
    'L_ht':'player_ht',
    'L_nationality':'player_nationality',
    'L_age':'player_age'
}, inplace=True)

In [58]:
df_transformed = pd.concat([df_winner, df_loser], ignore_index=True)
df_transformed.to_csv('modeling data/df1_workable.csv', index=False)

In [59]:
# df_transformed = df_transformed.sample(frac=1).reset_index(drop=True)

player_peak_ranking = df_transformed.groupby('player')['player_rank'].min().reset_index()
opponent_peak_ranking = df_transformed.groupby('opponent')['opponent_rank'].min().reset_index()
player_peak_ranking

player_updated = pd.merge(df_transformed, player_peak_ranking[['player', 'player_rank']], on='player', how='left')
opponent_updated = pd.merge(player_updated, opponent_peak_ranking[['opponent', 'opponent_rank']], on='opponent', how='left')
df1 = opponent_updated.copy()
df1.drop(columns=['player_rank_x', 'opponent_rank_x'], inplace=True)
df1 = df1.rename(columns={  'player_rank_y': 'player_rank',
                            'opponent_rank_y': 'opponent_rank'})
df1

,Location,Date,Round,player,opponent,player_hand,player_ht,player_nationality,player_age,opponent_hand,opponent_ht,opponent_nationality,opponent_age,win,player_rank,opponent_rank
0,Brisbane,01/01/2017,Round of 128,Thompson J.,Ymer E.,R,183.0,AUS,24.6,R,183.0,SWE,22.8,1,43.0,108.0
1,Brisbane,01/02/2017,Round of 128,Mahut N.,Robert S.,R,190.0,FRA,37.3,R,185.0,FRA,39.6,1,38.0,53.0
2,Brisbane,01/02/2017,Round of 128,Ferrer D.,Tomic B.,R,175.0,ESP,36.7,R,193.0,AUS,26.2,1,21.0,26.0
3,Brisbane,01/02/2017,Round of 128,Edmund K.,Escobedo E.,R,188.0,GBR,24.1,R,185.0,MEX,23.1,1,14.0,69.0
4,Brisbane,01/02/2017,Round of 128,Dimitrov G.,Johnson S.,R,188.0,BUL,27.6,R,188.0,USA,29.0,1,3.0,25.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35829,nitto-atp-finals,01/01/2017,Round Robin,Busta P.,Thiem D.,R,188.0,ESP,33.0,R,185.0,AUT,31.0,0,15.0,3.0
35830,nitto-atp-finals,01/01/2017,Round Robin,Cilic M.,Sock J.,R,198.0,CRO,36.0,R,191.0,USA,32.0,0,3.0,8.0
35831,nitto-atp-finals,01/01/2017,Round Robin,Busta P.,Dimitrov G.,R,188.0,ESP,33.0,R,191.0,BUL,33.0,0,15.0,3.0
35832,nitto-atp-finals,01/01/2017,Round Robin,Goffin D.,Dimitrov G.,R,180.0,BEL,34.0,R,191.0,BUL,33.0,0,7.0,3.0


In [60]:
df1_home_country = df1.copy()
df1_home_country['is_player_home_country'] = np.equal(df1_home_country['Location'], df1_home_country['player_nationality'])
df1_home_country['is_opponent_home_country'] = np.equal(df1_home_country['Location'], df1_home_country['opponent_nationality'])
df1_home_country.drop(columns=["Location",'player_nationality', 'opponent_nationality'], inplace=True)
df1_home_country


,Date,Round,player,opponent,player_hand,player_ht,player_age,opponent_hand,opponent_ht,opponent_age,win,player_rank,opponent_rank,is_player_home_country,is_opponent_home_country
0,01/01/2017,Round of 128,Thompson J.,Ymer E.,R,183.0,24.6,R,183.0,22.8,1,43.0,108.0,False,False
1,01/02/2017,Round of 128,Mahut N.,Robert S.,R,190.0,37.3,R,185.0,39.6,1,38.0,53.0,False,False
2,01/02/2017,Round of 128,Ferrer D.,Tomic B.,R,175.0,36.7,R,193.0,26.2,1,21.0,26.0,False,False
3,01/02/2017,Round of 128,Edmund K.,Escobedo E.,R,188.0,24.1,R,185.0,23.1,1,14.0,69.0,False,False
4,01/02/2017,Round of 128,Dimitrov G.,Johnson S.,R,188.0,27.6,R,188.0,29.0,1,3.0,25.0,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35829,01/01/2017,Round Robin,Busta P.,Thiem D.,R,188.0,33.0,R,185.0,31.0,0,15.0,3.0,False,False
35830,01/01/2017,Round Robin,Cilic M.,Sock J.,R,198.0,36.0,R,191.0,32.0,0,3.0,8.0,False,False
35831,01/01/2017,Round Robin,Busta P.,Dimitrov G.,R,188.0,33.0,R,191.0,33.0,0,15.0,3.0,False,False
35832,01/01/2017,Round Robin,Goffin D.,Dimitrov G.,R,180.0,34.0,R,191.0,33.0,0,7.0,3.0,False,False


In [61]:
# df1_home_country['Date'] = pd.to_datetime(df1_home_country['Date'])

# df1_home_country['year'] = df1_home_country['Date'].dt.year
# df1_home_country['month'] = df1_home_country['Date'].dt.month
# df1_home_country['day'] = df1_home_country['Date'].dt.day
# df1_home_country['dayofweek'] = df1_home_country['Date'].dt.dayofweek
# df1_home_country.drop(columns=["Date"], inplace=True)
# df1_home_country.to_csv('modeling data/df1_with_home.csv', index=False)

#df without location and nationality, only care about home advantage
df_home = pd.read_csv("modeling data/df1_with_home.csv")
df_home.columns

Index(['Round', 'player', 'opponent', 'player_hand', 'player_ht', 'player_age',
       'opponent_hand', 'opponent_ht', 'opponent_age', 'win', 'player_rank',
       'opponent_rank', 'is_player_home_country', 'is_opponent_home_country',
       'year', 'month', 'day', 'dayofweek'],
      dtype='object')

In [62]:
#encoding the date using sin and cos to preserve the cyclical nature of
df_date_encoded = df_home.copy()
df_date_encoded['month_sin'] = np.sin(2 * np.pi * df_date_encoded['month']/12)
df_date_encoded['month_cos'] = np.cos(2 * np.pi * df_date_encoded['month']/12)
df_date_encoded['dayofweek_sin'] = np.sin(2 * np.pi * df_date_encoded['dayofweek']/7)
df_date_encoded['dayofweek_cos'] = np.cos(2 * np.pi * df_date_encoded['dayofweek']/7)
df_date_encoded['day_sin'] = np.sin(2 * np.pi * df_date_encoded['day']/30.5)
df_date_encoded['day_cos'] = np.cos(2 * np.pi * df_date_encoded['day']/30.5)

df_date_encoded.drop(columns=["month","dayofweek", "day"], inplace=True)

In [63]:
#encoding rounds, no need to scale because the higher the round is, the more important it is
df_round_encoded = df_date_encoded.copy()
round_mapping = {
    '1st Round Qualifying': 1,
    '2nd Round Qualifying': 2,
    '3rd Round Qualifying': 3,
    'Round Robin': 4,
    'Round of 128': 5,
    'Round of 64': 6,
    'Round of 32': 7,
    'Round of 16': 8,
    'Quarterfinals': 9,
    'Semifinals': 10,
    'The Final': 11
}
df_round_encoded['Round'] = df_round_encoded['Round'].map(round_mapping)

In [64]:
#encoding home advantage or not for players and opponents
df_binary_encoded = df_round_encoded.copy()
df_binary_encoded['is_player_home_country'] = df_binary_encoded['is_player_home_country'].astype(int)
df_binary_encoded['is_opponent_home_country'] = df_binary_encoded['is_opponent_home_country'].astype(int)

In [65]:
#encoding players and opponents
df_player_encode = df_binary_encoded.copy()
# def encode(og_df):
#     df = og_df.copy()
#     for column in df.columns:
#         if column in ["player","opponent"]:
#             i = 0
#             for items in df[column].unique():
#                 df[column].replace({items: i}, inplace=True)
#                 i+=1
#     return df
# df_player_encode = encode(df_player_encode)

In [66]:
#integer encoding for handedness
df_hand_int_encoded = df_player_encode.copy()
hand_mapping = {
    'R': 1,
    'L': 2,
    'U': 0
}
df_hand_int_encoded['player_hand'] = df_hand_int_encoded['player_hand'].map(hand_mapping)
df_hand_int_encoded['opponent_hand'] = df_hand_int_encoded['opponent_hand'].map(hand_mapping)

In [67]:
#one-hot encoding for handedness
df_hand_1hot_encoded = df_player_encode.copy()

df_hand_1hot_encoded = pd.get_dummies(df_hand_1hot_encoded, columns=['player_hand'], prefix='player_hand')
df_hand_1hot_encoded = pd.get_dummies(df_hand_1hot_encoded, columns=['opponent_hand'], prefix='opponent_hand')

In [68]:
numeric_columns_1 = ['player_ht', 'opponent_ht']
numeric_columns_2 = ['player_age','opponent_age']
numeric_columns_3 = ['player_rank', 'opponent_rank']
numeric_columns_4 = ["year"]

def normalize(og_df, numeric_columns):
    df = og_df.copy()
    for column in numeric_columns:
        # Z-score normalization
        df[f'{column}_zscore'] = (df[column] - df[column].mean()) / df[column].std()
        # Min-max scaling
        df[f'{column}_minmax'] = (df[column] - df[column].min()) / (df[column].max() - df[column].min())
        # Decimal scaling
        max_length = max(df[column].apply(lambda x: len(str(np.floor(x))) if x > 0 else 1))
        df[f'{column}_decimal'] = df[column] / (10**max_length)
    return df

In [69]:
#no normalization (NN)
df_hand_int_encoded_NN = df_hand_int_encoded.copy()
df_hand_1hot_encoded_NN = df_hand_1hot_encoded.copy()

#normalize all
df_hand_int_N_all = normalize(df_hand_int_encoded_NN,numeric_columns_1+numeric_columns_2+numeric_columns_3+numeric_columns_4)
df_hand_1hot_N_all = normalize(df_hand_1hot_encoded_NN,numeric_columns_1+numeric_columns_2+numeric_columns_3+numeric_columns_4)

df_hand_int_N_all.drop(columns=numeric_columns_1+numeric_columns_2+numeric_columns_3+numeric_columns_4, inplace=True)
df_hand_1hot_N_all.drop(columns=numeric_columns_1+numeric_columns_2+numeric_columns_3+numeric_columns_4, inplace=True)

In [70]:
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

#split df into training and testing
def split_df(df):
    working_df = df.copy()
    X = working_df.drop(['win', 'player', 'opponent'], axis=1)  # Exclude non-numeric or target columns
    y = working_df['win']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
    return X_train, X_test, y_train, y_test

#train SVC model
def lin_svc_model(X_train, X_test, y_train, y_test):
    # Initialize the SVM classifier
    lsvm_model = LinearSVC(dual=True, max_iter=10000, random_state=1)
    lsvm_model.fit(X_train, y_train)
    y_pred_lsvm = lsvm_model.predict(X_test)
    print(classification_report(y_test, y_pred_lsvm))
    return y_test, y_pred_lsvm

#train SVC model
def svc_model(X_train, X_test, y_train, y_test):
    # Initialize the SVM classifier
    svm_model = SVC(kernel='linear', probability=True, random_state=1)
    svm_model.fit(X_train, y_train)
    y_pred_svm = svm_model.predict(X_test)
    print(classification_report(y_test, y_pred_svm))
    return y_test, y_pred_svm, svm_model, X_test

#train random forest model
def rfc_model(X_train, X_test, y_train, y_test):
    # Initialize the Random Forest classifier
    rf_model = RandomForestClassifier(n_estimators=100, random_state=1)
    rf_model.fit(X_train, y_train)
    y_pred_rf = rf_model.predict(X_test)
    print(classification_report(y_test, y_pred_rf))
    return y_test, y_pred_rf, rf_model, X_test

#train logistic regression model
def logReg_model(X_train, X_test, y_train, y_test):
    # Initialize the Logistic Regression classifier
    lr_model = LogisticRegression(random_state=1)
    lr_model.fit(X_train, y_train)
    y_pred_lr = lr_model.predict(X_test)
    print(classification_report(y_test, y_pred_lr))
    return y_test, y_pred_lr, lr_model, X_test

In [ ]:
# Calculate metrics
def calcMetric(y_test, y_pred):
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)

    print(f"Accuracy: {accuracy}")
    print(f"Precision: {precision}")
    print(f"Recall: {recall}")
    print(f"F1 Score: {f1}")

def calcMetrics(y_test, y_pred, model, X_test):
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_test, model.predict_proba(X_test)[:, 1])  # For binary classifications

    print(f"Accuracy: {accuracy}")
    print(f"Precision: {precision}")
    print(f"Recall: {recall}")
    print(f"F1 Score: {f1}")
    print(f"ROC-AUC Score: {roc_auc}")

In [71]:
def lin_svc_pipeline(og_df):
    print("Linear Support Vector Classifier")
    df = og_df.copy()
    X_train, X_test, y_train, y_test = split_df(df)
    y_test, y_pred= lin_svc_model(X_train, X_test, y_train, y_test)
    calcMetric(y_test, y_pred)

def svc_pipeline(og_df):
    print("Support Vector Classifier")
    df = og_df.copy()
    X_train, X_test, y_train, y_test = split_df(df)
    y_test, y_pred, model, X_test = svc_model(X_train, X_test, y_train, y_test)
    calcMetrics(y_test, y_pred, model, X_test)

def rfc_pipeline(og_df):
    print("Random Forest Classifier")
    df = og_df.copy()
    X_train, X_test, y_train, y_test = split_df(df)
    y_test, y_pred, model, X_test = rfc_model(X_train, X_test, y_train, y_test)
    calcMetrics(y_test, y_pred, model, X_test)
    
def logReg_pipeline(og_df):
    print("Logistic regression Classifier")
    df = og_df.copy()
    X_train, X_test, y_train, y_test = split_df(df)
    y_test, y_pred, model, X_test = logReg_model(X_train, X_test, y_train, y_test)
    calcMetrics(y_test, y_pred, model, X_test)

In [72]:
# dataframes = [df_hand_int_encoded_NN,df_hand_1hot_encoded_NN,df_hand_int_N_all,df_hand_1hot_N_all]
# dataframe_names = ["hand_int_encoded_NN","hand_1hot_encoded_NN","hand_int_N_all","hand_1hot_N_all"]

dataframes = [df_hand_int_N_all,df_hand_1hot_N_all]
dataframe_names = ["hand_int_N_all","hand_1hot_N_all"]

In [73]:
i = 0
for df in dataframes:
    print(" \n"+dataframe_names[i])
    lin_svc_pipeline(df)
    print(" \n"+dataframe_names[i])
    rfc_pipeline(df)
    print(" \n"+dataframe_names[i])
    logReg_pipeline(df)
    i+=1

 
hand_int_N_all
Linear Support Vector Classifier
              precision    recall  f1-score   support

           0       0.66      0.60      0.63      3681
           1       0.62      0.67      0.64      3486

    accuracy                           0.64      7167
   macro avg       0.64      0.64      0.64      7167
weighted avg       0.64      0.64      0.63      7167

Accuracy: 0.6351332496162969
Precision: 0.6150594451783356
Recall: 0.6678141135972461
F1 Score: 0.6403520836198597
 
hand_int_N_all
Random Forest Classifier
              precision    recall  f1-score   support

           0       0.71      0.69      0.70      3681
           1       0.68      0.70      0.69      3486

    accuracy                           0.69      7167
   macro avg       0.69      0.69      0.69      7167
weighted avg       0.70      0.69      0.69      7167

Accuracy: 0.6947118738663318
Precision: 0.6812849162011173
Recall: 0.6996557659208261
F1 Score: 0.6903481460515143
ROC-AUC Score: 0.7598224